# Tutorial 2.0: Imports, Paths, & Utils

This notebook contains all the Python imports required for Tutorial 2.

## 2.0.1 Python Imports

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

Please import:

In [6]:
import numpy
from IPython.display import HTML
from matplotlib import pyplot as plt
import itertools

from mbfit.utils.filesystem import Directory
from mbfit.utils.job_manager import SLURMJobManager, JobManager, LocalJobManager
from mbfit.calculator import QchemCalculator, OrcaCalculator, Psi4Calculator

from mbfit.system.system_like import BondType
from mbfit.system.definition import BasicFragmentDefinition, BasicCompoundDefinition, AtomDefinition, BondDefinition
from mbfit.system.initializer import VSEPRInitializer
from mbfit.system.descriptor import fragment_definition_to_xyz, compound_definition_to_xyz, compound_to_xyz, compound_to_commented_xyz
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system import BasicCompound
from mbfit.system.transformer import SystemTransformer
from mbfit.system.copier import CompoundCopier
from mbfit.system.transformer import make_scan
from mbfit.system.region import BasicMutableRegionManager
from mbfit.visualization import render_system, render_systems, render_overlayed_systems
from mbfit.visualization import plot_scan, plot_clusters
from mbfit.utils import constants

from mbfit.system.descriptor.energy_decomposition_comment_line_descriptor import EnergyDecompositionCommentLineDescriptor

from mbfit.calculator import calculate_energy, calculate_energies, minimize_structures, minimize_structure, calculate_vibrational_modes, write_vibrational_modes, read_vibrational_modes, calculate_nmer_energies, find_clusters
from mbfit.calculator import calculate_many_body_energies, calculate_many_body_energy, calculate_binding_energies, calculate_binding_energy
from mbfit.calculator import write_multiple_vibrational_modes, read_multiple_vibrational_modes

from mbfit.system.io import write_fragment_definition, write_definition, write_system, write_systems, read_fragment_definition, read_definition, read_system, read_systems, write_system_and_energy, write_systems_and_energies, read_system_and_energy, read_systems_and_energies
from mbfit.system.io import write_system_and_nmer_energies, write_systems_and_nmer_energies, read_system_and_nmer_energy, read_systems_and_nmer_energies
from mbfit.system.io import write_training_set_file

from mbfit.parametrization import calculate_polarizabilities_by_XDM, calculate_radii_by_XDM, calculate_C6_by_XDM, calculate_d6_from_radii, get_ESP_grid_points, calculate_ESP_at_grid_points, fit_charges_to_ESP

from mbfit.configurations import NormalModesConfigurationGenerator, get_pbmetad_parameters, perform_pbmetad_simulation

from mbfit.potential import calculate_Vphys_energies, calculate_Vphys_nmer_energies

from mbfit.sampling import perform_fps, split_pool, SplitStrategy, filter_configs_by_cutoff

from mbfit.polynomials_old import generate_polynomial

from mbfit.parametrization import generate_pip_parametizer, parametrize_pip, evaluate_pip, read_parameters, write_parameters

from mbfit.implementation import initialize_mbx, add_monomer_type_to_MBX, add_excluded_pairs, add_charges_to_MBX, add_polarizability_factors_to_MBX, add_polarizabilities_to_MBX, add_reciprocal_space_C6_to_MBX, add_C6_and_d6_to_MBX, compile_mbx, add_PIP_to_MBX


ImportError: cannot import name 'add_polarizability_factors_to_MBX' from 'mbfit.implementation' (/home/ebullvul/software/development/MB-Fit_versions/QDMS_2025/mbfit/implementation/__init__.py)

## 2.0.2 Util Functions

In [74]:
qchem_job_template="""
module load cpu/0.15.4  gcc/10.2.0  mvapich2/2.3.6 qchem/5.4
conda activate p310
cd ~/software/development/MB-Fit_versions/QDMS_2025
export MBFIT_HOME=$PWD
source sourceme.sh
cd -

export QCSCRATCH=/expanse/lustre/scratch/$USER/temp_project/qscratch
export QCLOCALSCR=/scratch/$USER/job_$SLURM_JOB_ID

"""

def get_qchem_job_manager(job_scratch_directory: Directory) -> JobManager:
    job_scratch_directory.create()
    return LocalJobManager()
    return SLURMJobManager(
            input_path=f"{job_scratch_directory}/job.in",
            output_path=f"{job_scratch_directory}/job.out",
            log_path=f"{job_scratch_directory}/job.log",
            job_path=f"{job_scratch_directory}/job.job",
            job_output_path=f"{job_scratch_directory}/job.stdout",
            job_error_path=f"{job_scratch_directory}/job.stderr",
            max_queue_size=10,
            job_name="slurm_job_manager",
            allocation="csd888",
            reservation=None,
            partition="shared",
            qos=None,
            max_walltime_minutes=60,
            nodes=1,
            tasks=1,
            processors_per_task=32,
            template=qchem_job_template
    )

In [75]:
orca_job_template="""
module load cpu/0.17.3b  gcc/10.2.0/npcyll4  openmpi/4.1.1/ygduf2r
module load orca/5.0.4
export ORCA_SCRATCH=/scratch/$USER/job_$SLURM_JOB_ID
conda activate p310
cd ~/software/development/MB-Fit_versions/QDMS_2025
export MBFIT_HOME=$PWD
source sourceme.sh
cd -
"""

def get_orca_job_manager(job_scratch_directory: Directory) -> JobManager:
    job_scratch_directory.create()
    return LocalJobManager()
    return SLURMJobManager(
            input_path=f"{job_scratch_directory}/job.in",
            output_path=f"{job_scratch_directory}/job.out",
            log_path=f"{job_scratch_directory}/job.log",
            job_path=f"{job_scratch_directory}/job.job",
            job_output_path=f"{job_scratch_directory}/job.stdout",
            job_error_path=f"{job_scratch_directory}/job.stderr",
            max_queue_size=10,
            job_name="slurm_job_manager",
            allocation="csd888",
            reservation=None,
            partition="shared",
            qos=None,
            max_walltime_minutes=60,
            nodes=1,
            tasks=32,
            processors_per_task=1,
            template=orca_job_template
    )